In [85]:
file_list_EU36_MPC001=open("/scratch/cnt0024/hmg2840/albert7a/EN4/profiles_files/NATL60-CJM165_EU_y2012-2013/EU36-MPC001/list_vert_EU36-MPC001.txt","r")
list_EU36_MPC001=file_list_EU36_MPC001.read()
list_list_EU_MPC001=list_EU36_MPC001.split("\n")

In [86]:
file_list_EU36_MPC002=open("/scratch/cnt0024/hmg2840/albert7a/EN4/profiles_files/NATL60-CJM165_EU_y2012-2013/EU36-MPC002/list_vert_EU36-MPC002.txt","r")
list_EU36_MPC002=file_list_EU36_MPC002.read()
prof_EU_MPC002=list_EU36_MPC002.split("\n")

In [87]:
list_commune_MPC001_MPC002=[]
for prof in list_list_EU_MPC001:
    if prof in prof_EU_MPC002:
        list_commune_MPC001_MPC002.append(prof)
        
    

In [88]:
file_list_EU36_MPC003=open("/scratch/cnt0024/hmg2840/albert7a/EN4/profiles_files/NATL60-CJM165_EU_y2012-2013/EU36-MPC003/list_vert_EU36-MPC003.txt","r")
list_EU36_MPC003=file_list_EU36_MPC003.read()
prof_EU_MPC003=list_EU36_MPC003.split("\n")

In [89]:
list_commune_MPC001_MPC002_MPC003=[]
for prof in list_commune_MPC001_MPC002:
    if prof in prof_EU_MPC003:
        list_commune_MPC001_MPC002_MPC003.append(prof)
        
    

In [90]:
print(len(list_commune_MPC001_MPC002_MPC003))

8905


In [91]:
list_prof_commune_MPC001_MPC002_MPC003=[]
for prof in list_commune_MPC001_MPC002_MPC003:
    list_prof_commune_MPC001_MPC002_MPC003.append(prof[4:])


In [92]:
print(str(list_prof_commune_MPC001_MPC002_MPC003[0]))

In [93]:
import numpy as np
import dask
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import datetime
import glob as glob
import time
from datetime import date
import io
import json


In [94]:
latmin,latmax,lonmin,lonmax=(32.6,65.6,-17,10)
datemin,datemax=('2012-10-01','2013-09-30')
jsonfile='NATL60-CJM165_EU_y2012-2013_final.json'

In [95]:
file='/scratch/cnt0024/hmg2840/albert7a/EN4/EN.4.2.0.f.profiles.g10.201201.nc'
ds=xr.open_dataset(file)

In [96]:
def select_profiles(latmin,latmax,lonmin,lonmax,datemin,datemax,jsonfile):
        ''' Identify all the EN4 profiles that fall within the selected zone and period
        '''
        ## Datasets

        #EN4
        diren4="/scratch/cnt0024/hmg2840/albert7a/EN4/"

        yearmin=datemin[0:4]
        monthmin=datemin[5:7]
        yearmax=datemax[0:4]
        monthmax=datemax[5:7]

        list_filesEN4=[]
        if yearmin == yearmax:
            for m in np.arange(int(monthmin),int(monthmax)+1):
                if m < 10:
                    list_filesEN4.append('EN.4.2.0.f.profiles.g10.'+yearmin+'0'+str(m)+'.nc')
                else:
                    list_filesEN4.append('EN.4.2.0.f.profiles.g10.'+yearmin+str(m)+'.nc')
        else:
            for m in np.arange(int(monthmin),13):
                if m < 10:
                    list_filesEN4.append('EN.4.2.0.f.profiles.g10.'+yearmin+'0'+str(m)+'.nc')
                else:
                    list_filesEN4.append('EN.4.2.0.f.profiles.g10.'+yearmin+str(m)+'.nc')
        if int(yearmin)+1 == int(yearmax):
            for m in np.arange(1,int(monthmax)+1):
                if m < 10:
                    list_filesEN4.append('EN.4.2.0.f.profiles.g10.'+yearmax+'0'+str(m)+'.nc')
                else:
                    list_filesEN4.append('EN.4.2.0.f.profiles.g10.'+yearmax+str(m)+'.nc')
        else:
            for year in np.arange(int(yearmin)+1,int(yearmax)):
                for m in np.arange(1,13):
                    if m < 10:
                        list_filesEN4.append('EN.4.2.0.f.profiles.g10.'+str(year)+'0'+str(m)+'.nc')
                    else:
                        list_filesEN4.append('EN.4.2.0.f.profiles.g10.'+str(year)+str(m)+'.nc')
        print(list_filesEN4)

        datetmin=pd.to_datetime(datemin)
        datetmax=pd.to_datetime(datemax)
        ttmin=datetime.datetime(int(datetmin.strftime('%Y')),int(datetmin.strftime('%m')),int(datetmin.strftime('%d')),int(datetmin.strftime('%H')),int(datetmin.strftime('%M')))
        ttmax=datetime.datetime(int(datetmax.strftime('%Y')),int(datetmax.strftime('%m')),int(datetmax.strftime('%d')),int(datetmax.strftime('%H')),int(datetmax.strftime('%M')))
        tsecmin=(ttmin-datetime.datetime(1958,1,1,0,0)).total_seconds()
        tsecmax=(ttmax-datetime.datetime(1958,1,1,0,0)).total_seconds()

        for f in np.arange(len(list_filesEN4)):
            fileEN4=list_filesEN4[f]
            print('Processing '+str(fileEN4))
            tfileEN4=diren4+fileEN4
            dsen4=xr.open_dataset(tfileEN4)
            laten4=dsen4['LATITUDE']
            lonen4=dsen4['LONGITUDE']
            dayen4=dsen4['JULD']
            refen4=dsen4['DC_REFERENCE']
                      
            indz=np.where((lonmin<lonen4.values)&(lonen4.values<lonmax)&(latmin<laten4.values)&(laten4.values<latmax))
            prof_zone=[]
            tsecen4z=[]
            for ref in np.arange(len(indz[0])):
                dateen4= pd.to_datetime(str(dayen4[indz[0][ref]].values))
                ten4 = datetime.datetime(int(dateen4.strftime('%Y')),int(dateen4.strftime('%m')),int(dateen4.strftime('%d')),int(dateen4.strftime('%H')),int(dateen4.strftime('%M')))
                tsecen4=(ten4-datetime.datetime(1958,1,1,0,0)).total_seconds()
                if (tsecen4 < tsecmax) & (tsecen4 > tsecmin):
                    prof_zone.append(indz[0][ref])
                    tsecen4z.append(tsecen4)
                    

            for ref in np.arange(len(prof_zone)):
                ref_str=str(refen4[prof_zone[ref]].values)[-16:-1]
                if ref_str in list_prof_commune_MPC001_MPC002_MPC003:
                    print(ref_str+' est bien dans la liste')
                    if 'dictyml' in locals():
                        up={str(refen4[prof_zone[ref]].values):{'reference':str(refen4[prof_zone[ref]].values),'file':fileEN4,'profile no':int(prof_zone[ref]),'latitude':float(laten4[prof_zone[ref]].values),'longitude':float(lonen4[prof_zone[ref]].values),'date':str(dayen4[prof_zone[ref]].values)}}
                        dictyml.update(up)
                    else:
                        dictyml={str(refen4[prof_zone[ref]].values):{'reference':str(refen4[prof_zone[ref]].values),'file':fileEN4,'profile no':int(prof_zone[ref]),'latitude':float(laten4[prof_zone[ref]].values),'longitude':float(lonen4[prof_zone[ref]].values),'date':str(dayen4[prof_zone[ref]].values)}}
                    
        with io.open(jsonfile, 'w', encoding='utf8') as outfile:
            outfile.write(str(json.dumps(dictyml, sort_keys=True,indent=4, separators=(',', ': '))))



In [97]:
select_profiles(latmin,latmax,lonmin,lonmax,datemin,datemax,jsonfile)

['EN.4.2.0.f.profiles.g10.201210.nc', 'EN.4.2.0.f.profiles.g10.201211.nc', 'EN.4.2.0.f.profiles.g10.201212.nc', 'EN.4.2.0.f.profiles.g10.201301.nc', 'EN.4.2.0.f.profiles.g10.201302.nc', 'EN.4.2.0.f.profiles.g10.201303.nc', 'EN.4.2.0.f.profiles.g10.201304.nc', 'EN.4.2.0.f.profiles.g10.201305.nc', 'EN.4.2.0.f.profiles.g10.201306.nc', 'EN.4.2.0.f.profiles.g10.201307.nc', 'EN.4.2.0.f.profiles.g10.201308.nc', 'EN.4.2.0.f.profiles.g10.201309.nc']
Processing EN.4.2.0.f.profiles.g10.201210.nc
A20121001-03180 est bien dans la liste
A20121001-03183 est bien dans la liste
A20121001-03275 est bien dans la liste
A20121001-03276 est bien dans la liste
A20121001-03285 est bien dans la liste
A20121002-03316 est bien dans la liste
A20121002-03350 est bien dans la liste
A20121002-03375 est bien dans la liste
A20121002-03380 est bien dans la liste
A20121002-03401 est bien dans la liste
A20121003-03431 est bien dans la liste
A20121003-03483 est bien dans la liste
A20121003-03505 est bien dans la liste
A20

A20121012-43647 est bien dans la liste
A20121013-43648 est bien dans la liste
A20121014-43649 est bien dans la liste
A20121012-43650 est bien dans la liste
A20121013-43651 est bien dans la liste
A20121014-43652 est bien dans la liste
A20121015-44452 est bien dans la liste
A20121016-44453 est bien dans la liste
A20121016-44454 est bien dans la liste
A20121017-45153 est bien dans la liste
A20121018-45154 est bien dans la liste
A20121017-45156 est bien dans la liste
A20121018-45157 est bien dans la liste
A20121019-45836 est bien dans la liste
A20121020-45837 est bien dans la liste
A20121021-45838 est bien dans la liste
A20121019-45847 est bien dans la liste
A20121020-45848 est bien dans la liste
A20121021-45849 est bien dans la liste
A20121023-46578 est bien dans la liste
A20121023-46582 est bien dans la liste
A20121024-47129 est bien dans la liste
A20121024-47130 est bien dans la liste
A20121001-69083 est bien dans la liste
A20121001-69092 est bien dans la liste
A20121001-69104 est bien 

A20121024-70710 est bien dans la liste
A20121024-70715 est bien dans la liste
A20121024-70742 est bien dans la liste
A20121024-70756 est bien dans la liste
A20121024-70761 est bien dans la liste
A20121024-70773 est bien dans la liste
A20121025-70809 est bien dans la liste
A20121026-70860 est bien dans la liste
A20121026-70880 est bien dans la liste
A20121027-70917 est bien dans la liste
A20121027-70921 est bien dans la liste
A20121027-70929 est bien dans la liste
A20121027-70937 est bien dans la liste
A20121027-70938 est bien dans la liste
A20121027-70941 est bien dans la liste
A20121027-70950 est bien dans la liste
A20121027-70953 est bien dans la liste
A20121027-70958 est bien dans la liste
A20121027-70963 est bien dans la liste
A20121027-70967 est bien dans la liste
A20121027-70972 est bien dans la liste
A20121027-70974 est bien dans la liste
A20121028-70984 est bien dans la liste
A20121028-70992 est bien dans la liste
A20121028-71002 est bien dans la liste
A20121028-71018 est bien 

A20121129-06356 est bien dans la liste
A20121129-06387 est bien dans la liste
A20121129-06407 est bien dans la liste
A20121129-06412 est bien dans la liste
A20121129-06414 est bien dans la liste
A20121130-06446 est bien dans la liste
A20121130-06457 est bien dans la liste
A20121130-06524 est bien dans la liste
A20121130-06546 est bien dans la liste
A20121130-06550 est bien dans la liste
A20121103-39986 est bien dans la liste
A20121101-40967 est bien dans la liste
A20121102-40968 est bien dans la liste
A20121104-40969 est bien dans la liste
A20121108-40972 est bien dans la liste
A20121109-40973 est bien dans la liste
A20121110-40974 est bien dans la liste
A20121111-40975 est bien dans la liste
A20121112-40976 est bien dans la liste
A20121101-40979 est bien dans la liste
A20121102-40980 est bien dans la liste
A20121103-40981 est bien dans la liste
A20121104-40982 est bien dans la liste
A20121105-40983 est bien dans la liste
A20121108-40986 est bien dans la liste
A20121109-40987 est bien 

A20121213-04607 est bien dans la liste
A20121214-04625 est bien dans la liste
A20121214-04648 est bien dans la liste
A20121214-04679 est bien dans la liste
A20121214-04682 est bien dans la liste
A20121214-04701 est bien dans la liste
A20121215-04727 est bien dans la liste
A20121215-04728 est bien dans la liste
A20121215-04731 est bien dans la liste
A20121215-04733 est bien dans la liste
A20121215-04781 est bien dans la liste
A20121215-04805 est bien dans la liste
A20121216-04869 est bien dans la liste
A20121216-04890 est bien dans la liste
A20121216-04906 est bien dans la liste
A20121217-04929 est bien dans la liste
A20121217-04932 est bien dans la liste
A20121219-05254 est bien dans la liste
A20121220-05277 est bien dans la liste
A20121220-05292 est bien dans la liste
A20121220-05366 est bien dans la liste
A20121220-05380 est bien dans la liste
A20121220-05381 est bien dans la liste
A20121221-05402 est bien dans la liste
A20121221-05413 est bien dans la liste
A20121221-05473 est bien 

...

A20130814-79286 est bien dans la liste
A20130830-79467 est bien dans la liste
A20130801-79959 est bien dans la liste
A20130801-79964 est bien dans la liste
A20130801-79966 est bien dans la liste
A20130801-79967 est bien dans la liste
A20130801-79970 est bien dans la liste
A20130801-79971 est bien dans la liste
A20130801-79977 est bien dans la liste
A20130801-79978 est bien dans la liste
A20130801-79980 est bien dans la liste
A20130802-80064 est bien dans la liste
A20130802-80066 est bien dans la liste
A20130803-80073 est bien dans la liste
A20130803-80087 est bien dans la liste
A20130803-80094 est bien dans la liste
A20130803-80102 est bien dans la liste
A20130803-80111 est bien dans la liste
A20130803-80117 est bien dans la liste
A20130803-80127 est bien dans la liste
A20130803-80144 est bien dans la liste
A20130803-80148 est bien dans la liste
A20130803-80154 est bien dans la liste
Processing EN.4.2.0.f.profiles.g10.201309.nc
A20130901-03345 est bien dans la liste
A20130901-03375 est

A20130929-41048 est bien dans la liste
A20130920-41064 est bien dans la liste
A20130926-41065 est bien dans la liste
A20130929-41067 est bien dans la liste
A20130923-41069 est bien dans la liste
A20130923-41070 est bien dans la liste
A20130928-41071 est bien dans la liste
A20130929-41072 est bien dans la liste
A20130921-41084 est bien dans la liste
A20130921-41086 est bien dans la liste
A20130927-41089 est bien dans la liste
A20130928-41101 est bien dans la liste
A20130927-41103 est bien dans la liste
A20130917-41109 est bien dans la liste
A20130928-41114 est bien dans la liste
A20130928-41115 est bien dans la liste
A20130928-41116 est bien dans la liste
A20130928-41117 est bien dans la liste
A20130901-41177 est bien dans la liste
A20130901-41178 est bien dans la liste
A20130901-41179 est bien dans la liste
A20130901-41180 est bien dans la liste
A20130901-41181 est bien dans la liste
A20130901-41182 est bien dans la liste
A20130902-41183 est bien dans la liste
A20130902-41184 est bien 

A20130925-63880 est bien dans la liste
A20130925-63889 est bien dans la liste
A20130925-63896 est bien dans la liste
A20130925-63902 est bien dans la liste
A20130925-63904 est bien dans la liste
A20130925-63910 est bien dans la liste
A20130925-63914 est bien dans la liste
A20130926-63946 est bien dans la liste
A20130926-63962 est bien dans la liste
A20130926-63970 est bien dans la liste
A20130927-64015 est bien dans la liste
A20130928-64077 est bien dans la liste
A20130928-64096 est bien dans la liste
A20130929-64150 est bien dans la liste
A20130929-64177 est bien dans la liste
A20130901-65377 est bien dans la liste
A20130901-65381 est bien dans la liste
A20130901-65385 est bien dans la liste
A20130901-65394 est bien dans la liste
A20130901-65398 est bien dans la liste
A20130901-65400 est bien dans la liste
A20130901-65411 est bien dans la liste
A20130901-65417 est bien dans la liste
A20130901-65424 est bien dans la liste
A20130901-65432 est bien dans la liste
A20130901-65435 est bien 

A20130911-67783 est bien dans la liste
A20130912-67810 est bien dans la liste
A20130912-67841 est bien dans la liste
A20130912-67842 est bien dans la liste
A20130912-67905 est bien dans la liste
A20130912-67936 est bien dans la liste
A20130912-67997 est bien dans la liste
A20130912-67998 est bien dans la liste
A20130912-68033 est bien dans la liste
A20130912-68034 est bien dans la liste
A20130912-68090 est bien dans la liste
A20130912-68091 est bien dans la liste
A20130912-68129 est bien dans la liste
A20130912-68131 est bien dans la liste
A20130912-68153 est bien dans la liste
A20130912-68200 est bien dans la liste
A20130912-68201 est bien dans la liste
A20130912-68243 est bien dans la liste
A20130912-68245 est bien dans la liste
A20130912-68270 est bien dans la liste
A20130912-68357 est bien dans la liste
A20130912-68358 est bien dans la liste
A20130912-68451 est bien dans la liste
A20130913-68597 est bien dans la liste
A20130913-68598 est bien dans la liste
A20130913-68646 est bien 

A20130924-80865 est bien dans la liste
A20130924-80926 est bien dans la liste
A20130924-80927 est bien dans la liste
A20130924-80976 est bien dans la liste
A20130924-81061 est bien dans la liste
A20130924-81092 est bien dans la liste
A20130924-81119 est bien dans la liste
A20130924-81120 est bien dans la liste
A20130924-81189 est bien dans la liste
A20130924-81244 est bien dans la liste
A20130924-81286 est bien dans la liste
A20130924-81287 est bien dans la liste
A20130924-81333 est bien dans la liste
A20130924-81334 est bien dans la liste
A20130924-81401 est bien dans la liste
A20130924-81473 est bien dans la liste
A20130924-81508 est bien dans la liste
A20130924-81566 est bien dans la liste
A20130924-81613 est bien dans la liste
A20130924-81673 est bien dans la liste
A20130925-81720 est bien dans la liste
A20130925-81761 est bien dans la liste
A20130925-81797 est bien dans la liste
A20130925-81899 est bien dans la liste
A20130925-81967 est bien dans la liste
A20130925-82060 est bien 

In [98]:
!ls $jsonfile

NATL60-CJM165_EU_y2012-2013_final.json
